In [1]:
%%local 
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0,com.esri.geometry:esri-geometry-api:1.2.1,commons-io:commons-io:2.6,org.apache.spark:spark-streaming_2.11:2.2.0,org.apache.spark:spark-sql_2.11:2.2.0",
        "spark.jars":"/home/isam/Desktop/zeppelin-packages/magellan-1.0.5-s_2.11.jar",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11",
        "spark.dynamicAllocation.enabled": false
    }
}

In [2]:
/**
 * @Description: SAOS integrated with ApproxSPE 
 * @author: Isam Al Jawarneh
 * @date: 24/10/2021
 */


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
//import org.apache.spark.util.random.XORShiftRandom
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}

import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{
  DoubleType,
  StringType,
  StructField,
  StructType
}
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions.{collect_list, collect_set}
import org.apache.spark.sql.SQLContext
import org.apache.log4j.{Level, Logger}
import scala.collection.mutable
import scala.concurrent.duration.Duration
import java.io.{BufferedWriter, FileWriter}
import org.apache.commons.io.FileUtils
import java.io.File
import scala.collection.mutable.ListBuffer
import java.time.Instant
//import org.apache.spark.util.CollectionAccumulator
import org.apache.spark.sql.DataFrame
import org.apache.kafka.clients.consumer.ConsumerRecord
import org.apache.kafka.common.serialization.StringDeserializer


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}
import org.apache.spark.s

In [4]:
val schemaNYCshort = StructType(Array(
    StructField("id", StringType, false),
    StructField("lat", DoubleType, false),
    StructField("lon", DoubleType, false),
    StructField("time", StringType, false),
    StructField("speed", DoubleType, false)))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schemaNYCshort: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,false), StructField(lat,DoubleType,false), StructField(lon,DoubleType,false), StructField(time,StringType,false), StructField(speed,DoubleType,false))


In [5]:
val geohashUDF = udf{(curve: Seq[ZOrderCurve]) => curve.map(_.toBase32())}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

geohashUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(ArrayType(org.apache.spark.sql.types.ZOrderCurveUDT@e7320b8,true))))


In [7]:
val trips =spark.read.format("csv").option("header", "true").schema(schemaNYCshort).csv("/home/isam/Desktop/time-series/data/china/trips/").withColumn("point", point($"lat",$"lon"))
val ridesGeohashed = trips.withColumn("index", $"point" index  30).withColumn("geohashArray1", geohashUDF($"index.curve"))//.select($"id", $"vendorId", $"point",$"geohashArray",$"Trip_distance")
val explodedRidesGeohashed = ridesGeohashed.explode("geohashArray1", "geohash") { a: mutable.WrappedArray[String] => a }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trips: org.apache.spark.sql.DataFrame = [id: string, lat: double ... 4 more fields]
ridesGeohashed: org.apache.spark.sql.DataFrame = [id: string, lat: double ... 6 more fields]
explodedRidesGeohashed: org.apache.spark.sql.DataFrame = [id: string, lat: double ... 7 more fields]


In [8]:
explodedRidesGeohashed.show(2,false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+------------------+------------------------+-----+---------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------+
|id |lat               |lon               |time                    |speed|point                                        |index                                                                                                                                                  |geohashArray1|geohash|
+---+------------------+------------------+------------------------+-----+---------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------+
|0  |114.03179899999999|22.524798999999998|2014-10-22T02:54:30.000Z|42.0 |Point(114.03179899999999, 22.524798999999

In [9]:
explodedRidesGeohashed.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: Long = 1155653


In [13]:
explodedRidesGeohashed.select($"id",$"lon",$"lat",$"geohash",$"speed",$"time").coalesce(1).write.format("com.databricks.spark.csv").option("header",true).save("/home/isam/Desktop/time-series/data/china/ShenzhenTripsGeo.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
import org.apache.spark.sql.functions._
explodedRidesGeohashed.agg(skewness(explodedRidesGeohashed("speed"))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
+------------------+
|   skewness(speed)|
+------------------+
|1.3405139822371657|
+------------------+

